In [2]:
# get secret key
from mylibrary.secrets import app_id, app_key, orig_name, dest_name, orig_pc, dest_pc
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

/Users/robinlinacre/anaconda/lib/python2.7/site-packages/sqlalchemy/dialects/postgresql/base.py:2505: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [2]:
# RUNNING THIS WILL DELETE ALL YOUR JOURNEYTIME RESULTS!!!

# for name in (orig_name, dest_name):
#     sql = """
#     drop table if exists tt_gh.stations_journeytime_{name};
    
#     create table tt_gh.stations_journeytime_{name} as 
#     select nlc, icscode, station_name  from tt_gh.all_stations where london_or_gb = 'gb' and icscode_status= 'ok';
#     """.format(name = name)
#     cur = conn.cursor()
#     cur.execute(sql)
#     conn.commit()

#     sql = """
#     ALTER TABLE tt_gh.stations_journeytime_{name} 
#     ADD tfl_request text, ADD tfl_response json,
#     ADD querydict json,
#     ADD journey_time int,
#     ADD tfl_message text,
#     ADD london_mainline_station_ics text,
#     ADD london_mainline_station_text text;
#     ALTER TABLE tt_gh.stations_journeytime_{name} ADD PRIMARY KEY (nlc);

#     """.format(name=name)
#     cur = conn.cursor()
#     cur.execute(sql)
#     conn.commit()

In [3]:
# Dump all the json to the database first.  Then process it later
# Iterate through adding icscode
import requests
import pandas as pd
import json

from mylibrary.tfl_helpers import get_journeyplanner_raw_content, status_of_tfl_response_journey, get_journeyplanner_results

In [ ]:
from sqlalchemy import or_
Stations_orig = Automapped_Base.classes.stations_journeytime_stpauls
Stations_dest = Automapped_Base.classes.stations_journeytime_camden

for Stations in (Stations_orig, Stations_dest):
    nlcs  = session.query(Stations.nlc).filter(or_(Stations.tfl_message != 'ok',Stations.tfl_message == None)).all()
#     nlcs  = session.query(Stations.nlc).filter(Stations.tfl_message == None).all()
    for nlc in nlcs:
        station = session.query(Stations).filter(Stations.nlc == nlc).one()
        
        for d in param_dicts:
            d["from"] = station.icscode
            if "stpauls" in Stations.__table__.name:
                d["to"] = orig_pc.replace(" ","")
            if "camden" in Stations.__table__.name:
                d["to"] = dest_pc.replace(" ","")
                
        journey_object = get_journeyplanner_results(param_dicts)

        station.tfl_request = journey_object["request_url"]
        station.tfl_response = journey_object["tfl_response"]
        station.querydict = journey_object["query_dict"]
        station.tfl_message = journey_object["tfl_message"]

        session.add(station)
        session.commit()

In [4]:
# Multithread 
import threading
num_threads = 10

from sqlalchemy import or_


from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres:@localhost:5432/postgres') 

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, MetaData



Stations_dest = Automapped_Base.classes.stations_journeytime_camden
nlcs  = session.query(Stations_dest.nlc).all()

counter = 1

def new_thread():
    global counter
    engine = create_engine('postgresql://postgres:@localhost:5432/postgres') 
    m = MetaData(schema='tt_gh')
    Automapped_Base = automap_base(bind=engine,metadata=m)
    Automapped_Base.prepare(engine, reflect=True)
    session = Session(engine)

    Stations_dest = Automapped_Base.classes.stations_journeytime_camden
    
    param_dict = {
    "to": orig_pc.replace(" ",""),
    "id": app_id,
    "key": app_key,
    "time": "1000", 
    "date": "20160517",
    "nationalsearch": True
    }

    param_dicts = []
    param_dicts.append(param_dict)

    d1 = param_dict.copy()
    d1["time"] = "1500"
    d1["date"] = "20160518"
    param_dicts.append(d1)

    d2 = param_dict.copy()
    d2["time"] = "1600"
    d2["date"] = "20160519"
    param_dicts.append(d2)

    while len(nlcs) > 0:
        nlc = nlcs.pop()
        station = session.query(Stations_dest).filter(Stations_dest.nlc == nlc).one()
        
        for d in param_dicts:
            d["from"] = station.icscode
            if "stpauls" in Stations_dest.__table__.name:
                d["to"] = orig_pc.replace(" ","")
            if "camden" in Stations_dest.__table__.name:
                d["to"] = dest_pc.replace(" ","")
                
        journey_object = get_journeyplanner_results(param_dicts)

        station.tfl_request = journey_object["request_url"]
        station.tfl_response = journey_object["tfl_response"]
        station.querydict = journey_object["query_dict"]
        station.tfl_message = journey_object["tfl_message"]

        session.add(station)
        session.commit()
        
        counter +=1
        if counter % 10 == 0:
            print counter
 
my_threads = []

for i in range(num_threads):
    t = threading.Thread(target=new_thread)
    my_threads.append(t)
    t.start() #start this thread

for x in my_threads:
    x.join()

10
20
30
Traceback (most recent call last):
  File "mylibrary/tfl_helpers.py", line 30, in status_of_tfl_response_journey
    parsed_response = json.loads(content)
  File "/Users/robinlinacre/anaconda/lib/python2.7/json/__init__.py", line 339, in loads
    return _default_decoder.decode(s)
  File "/Users/robinlinacre/anaconda/lib/python2.7/json/decoder.py", line 364, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/robinlinacre/anaconda/lib/python2.7/json/decoder.py", line 382, in raw_decode
    raise ValueError("No JSON object could be decoded")
ValueError: No JSON object could be decoded

Traceback (most recent call last):
  File "mylibrary/tfl_helpers.py", line 30, in status_of_tfl_response_journey
    parsed_response = json.loads(content)
  File "/Users/robinlinacre/anaconda/lib/python2.7/json/__init__.py", line 339, in loads
    return _default_decoder.decode(s)
  File "/Users/robinlinacre/anaconda/lib/python2.7/json/decoder.py", line 364, in decode
 

/Users/robinlinacre/anaconda/lib/python2.7/site-packages/sqlalchemy/sql/type_api.py:279: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return x == y
